In [1]:
!pip -q install grpcio grpcio-reflection


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
from caikit_nlp_client import HttpClient

model_endpoint_url = "https://caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com"
embedding_model_name = "all-MiniLM-L12-v2-caikit"

http_client = HttpClient(model_endpoint_url, verify=False)
token = '''\
Bearer eyJhbGciOiJSUzI1NiIsImtpZCI6IjZnRXdDUGJfbE44Q29tTWNER0ZtRWN2RE52REt2MlRPTE95TE1USGRYTzQifQ.eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZX
Rlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJqYXJ5LXdiIiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9zZWNyZXQubmFtZSI6ImRlZmF1bHQtbmFtZS1jYWlraXQtbm9taWM1LXNhIiwia3ViZXJu
ZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9zZXJ2aWNlLWFjY291bnQubmFtZSI6ImNhaWtpdC1ub21pYzUtc2EiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlcnZpY2UtYWNjb3VudC51aWQiOiIxMjY1ZTQ3ZS
01NzhjLTRiOTEtYjdlNy0yODRhNTNlYjk5YTkiLCJzdWIiOiJzeXN0ZW06c2VydmljZWFjY291bnQ6amFyeS13YjpjYWlraXQtbm9taWM1LXNhIn0.J6MrigUYdv9IgXJvEbZW8u14NTLQUc5bhvLsp3OHV-NYVlK2_XTO
SqwSoP3C4s6iNftDby0_5wN43dht5WLlIHUwwfcDjK_pmOKXZhy-WlxJsKkwL6H-aohIlLsBcL9-SHchztRmPU7xTpNI087IuG4AeX75mTEEWRqADuFo4L1cKgNeBHDJqDXBlYoOs1SSNFb65phb0SNAu48Qo2cPc0YSK_
l23VQg_ZdQYAyXMtgY6AOB0_xEk4busgcAsqKZ7hI6D0W7tAjDmXJTk4BCwKCtV8IAcGuP_XUkxTOAfWoJUzmDIVviqvFszleIV3E1NpoCunVXDhmPpTUs__weSd8_S6fK3H-G8sJ7s8ZeZUUOdZ4wfP0Jgk2XdkX6FinC
E_wHLHBtbJRFHlZgRt2YYS0OYivaVlnfjmi4b3EL2iD-pnqwg9sgYiEx0oNLzw5Dyp-nHECfK8ijFI2dvNVJpvTiezDLq0oF8yNAJGgNU518F526ljpUKSD6levqTVpTBbdYFZtE3hkJInQY2yjlcGg8gxrhd-A3bGov_b
EDLLeOXq4IZVDvE-VvSLyAklWxeBZMY9aQPKecgnLL8se8Br_k7WnIxUOOmmJY4gTU-t5j_C_poutvGQfez1Avw1aEZubXa7M5KzjwM2wCUtS17lM3FMLElejTYAEKLpGdaa0
'''.replace('\n', '')

http_client.embedding(
  token, embedding_model_name, "Sample Text"
)



/opt/app-root/lib64/python3.9/site-packages/google/protobuf/runtime_version.py:112: UserWarning: Protobuf gencode version 5.27.2 is older than the runtime version 5.28.1 at grpc_reflection/v1alpha/reflection.proto. Please avoid checked-in Protobuf gencode that can be obsolete.
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'caikit-nomic5-jary-wb.apps.jary-intel-opea.51ty.p1.openshiftapps.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'result': {'data': {'values': [0.008481573313474655,
    0.038828518241643906,
    0.0009980080649256706,
    -0.050852347165346146,
    0.044094230979681015,
    -0.025899769738316536,
    0.03840351104736328,
    0.022538907825946808,
    -0.025817468762397766,
    -0.01586952619254589,
    -0.00116447894833982,
    0.040396835654973984,
    0.06008037179708481,
    -0.08361504971981049,
    -0.044891487807035446,
    0.0005722575588151813,
    0.09311066567897797,
    0.028177019208669662,
    -0.019082706421613693,
    0.050123099237680435,
    0.01836974173784256,
    -0.03967832401394844,
    0.08204962313175201,
    -0.03393074497580528,
    -0.02802625298500061,
    0.015732640400528908,
    -0.015777641907334328,
    0.03854375705122948,
    -0.0029222711455076933,
    -0.07307330518960953,
    -0.0203690342605114,
    0.08696506172418594,
    0.09249444305896759,
    0.02484230324625969,
    -0.05890114605426788,
    -0.015400202944874763,
    -0.00025278303655795753,
    0.